In [1]:
# # On Google colab:

# # Data
# !mkdir data
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# !mv input.txt data/shakespeare.txt

# # Module
# !pip install git+https://github.com/shuiruge/nanogpt.git

In [2]:
# Locally:
import sys
sys.path.append('../nanogpt')

In [3]:
import numpy as np
import tensorflow as tf
from keras.layers import Layer, Dense
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping
from dataclasses import dataclass
from typing import List

# from nanogpt.utils import (
from utils import (
    CharacterTokenizer, LanguageModelDataGenerator, TokPosEmbedding, FeedForward,
    luong_attention, MultiHeadWrapper,
)

tf.random.set_seed(42)

2024-03-16 20:50:50.418813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
with open('data/shakespeare.txt', 'r') as f:
    corpus = ''.join(f.readlines())
print(corpus[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
tokenizer = CharacterTokenizer(corpus)
tokenizer.vocab_size

65

In [6]:
seq_len = 20
data = LanguageModelDataGenerator(tokenizer.encode(corpus))

contexts = []
targets = []
while True:
    try:
        context, target = data(seq_len, False)
        contexts.append(context)
        targets.append(target)
    except StopIteration:
        break
contexts = np.stack(contexts).astype('int64')
targets = np.asarray(targets, 'int64')

In [7]:
class VanillaSelfAttention(Layer):
    """A vanilla version of self-attention.
    
    It is called vanilla because it has no trainable variables at all! The
    function of this kind of self-attention is just communicating between
    each token (or each node if there is an image of community in your mind).
    So, it has no resposibility for computation, which is completely left to
    feed-forward layers.

    In this communication viewpoint, the multi-head means the multi-channel
    of communication. Each channel propagates one kind of information. And
    different kinds of information are sent to different target nodes.

    Args:
        num_heads: int
    """

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads

        self.multihead = MultiHeadWrapper(self.num_heads)

    def call(self, x, return_weights=False):
        """
        Args:
            x: tf.Tensor
                Shape [..., seq_len, dim]
            return_weights: bool
                If return the attention weights. Defaults to False.

        Returns: tf.Tensor
            The same shape and dtype as the x.
        """
        query = self.multihead.split_heads(x)
        key = self.multihead.split_heads(x)
        value = self.multihead.split_heads(x)

        output, attention_weights = luong_attention(query, key, value)
        output = self.multihead.concat_heads(output)
        return (output, attention_weights) if return_weights else output

In [8]:
@dataclass
class GPTConfig:
    vocab_size: int
    seq_len: int
    embed_dim: int
    model_dim: int
    num_heads: int
    ffd_hidden_units: List[int]
    num_trans_blocks: int


class VanillaGPT(Model):
    """Build a vanilla GPT with vanilla self-attention."""

    def __init__(self, cfg: GPTConfig, **kwargs):
        super().__init__(**kwargs)
        self.cfg = cfg

        self.embedding_layer = TokPosEmbedding(
            cfg.vocab_size, cfg.seq_len, cfg.embed_dim)

        # The so-called transformer-blocks.
        self.trans_blocks = []
        for _ in range(cfg.num_trans_blocks):
            # As we have discussed in the docstring of VanillaSelfAttention,
            # the task of computation is left to feed-forward layers. So,
            # for communication:
            self.trans_blocks.append(
                VanillaSelfAttention(cfg.num_heads)
            )
            # and for computation:
            self.trans_blocks.append(
                FeedForward(cfg.ffd_hidden_units, cfg.model_dim)
            )

        self.output_layer = Dense(cfg.vocab_size)

    def call(self, x):
        x = self.embedding_layer(x)
        for layer in self.trans_blocks:
            x = layer(x)
        # We only use the last sequence element for output.
        # It is such a waste!
        # And why it is the last one? Why not the others? Maybe some of the
        # others out-performs the last.
        x = x[:, -1, :]
        x = self.output_layer(x)
        return x

    def generate(self, init_token_ids, num_new_tokens, T):
        """Generates new tokens from the initial.

        The "temperature" T controls the randomness, as in the Boltzmann
        distributions.

        Args:
            init_token_ids: List[int]
            num_new_tokens: int
            T: float

        Returns: List[int]
            It also includes the initial token-IDs. So, the length is the
            `len(initial_token_ids) + 
        """
        init_token_ids = tf.convert_to_tensor(init_token_ids)

        # Add batch_size for matching the input shape of `self.call`.
        # [1, len(init_token_ids)]
        token_ids = tf.expand_dims(init_token_ids, axis=0)

        for _ in range(num_new_tokens):
            # [1, vocab_size]
            logits = self(token_ids[:, -self.cfg.seq_len:])
            # [1, 1]
            next_token_id = tf.random.categorical(logits/T, 1)
            token_ids = tf.concat([token_ids, next_token_id], axis=1)

        # Drop the batch_size
        token_ids = tf.squeeze(token_ids, axis=0)
        return token_ids

In [9]:
# Andrej Karpathy's configuration.
# See: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=6068s, at 1:40:30.
# This configuration will get a minimal validation loss about 1.48.
# And this configuration is not for our vanilla GPT, but Andrej's nano GPT,
# which is much more complicated than ours.
# cfg = GPTConfig(tokenizer.vocab_size, seq_len,
#                 embed_dim=384,
#                 model_dim=384,
#                 num_heads=6,
#                 ffd_hidden_units=[4*384],
#                 num_trans_blocks=6)

# We try a much much smaller one.
# This configuration will get a minimal validation loss about 1.77.
cfg = GPTConfig(tokenizer.vocab_size, seq_len,
                embed_dim=64,
                model_dim=64,
                num_heads=4,
                ffd_hidden_units=[256],
                num_trans_blocks=2)

model = VanillaGPT(cfg)
model.build([None, seq_len])
model.summary()

Model: "vanilla_gpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tok_pos_embedding (TokPosE  multiple                  5440      
 mbedding)                                                       
                                                                 
 vanilla_self_attention (Va  multiple                  0         
 nillaSelfAttention)                                             
                                                                 
 feed_forward (FeedForward)  multiple                  33216     
                                                                 
 vanilla_self_attention_1 (  multiple                  0         
 VanillaSelfAttention)                                           
                                                                 
 feed_forward_1 (FeedForwar  multiple                  33216     
 d)                                                    

In [10]:
model.compile(
    optimizer=AdamW(),
    loss=SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(
    x=contexts,
    y=targets,
    batch_size=64,
    validation_split=0.1,
    # The epochs argument shall be as large as possible. And we control the
    # true epochs by early-stopping.
    epochs=100,
    callbacks=[EarlyStopping()]
)
# For our smaller configuration, the training will overfit after epoch 9.

Epoch 1/100
15685/15685 [==============================] - 142s 9ms/step - loss: 2.9399 - val_loss: 2.6986
Epoch 2/100
15685/15685 [==============================] - 139s 9ms/step - loss: 2.0457 - val_loss: 1.9839
Epoch 3/100
15685/15685 [==============================] - 142s 9ms/step - loss: 1.7705 - val_loss: 1.8922
Epoch 4/100
15685/15685 [==============================] - 142s 9ms/step - loss: 1.6799 - val_loss: 1.8313
Epoch 5/100
15685/15685 [==============================] - 153s 10ms/step - loss: 1.6315 - val_loss: 1.7912
Epoch 6/100
15685/15685 [==============================] - 172s 11ms/step - loss: 1.5993 - val_loss: 1.7874
Epoch 7/100
15685/15685 [==============================] - 161s 10ms/step - loss: 1.5768 - val_loss: 1.7691
Epoch 8/100
15685/15685 [==============================] - 154s 10ms/step - loss: 1.5592 - val_loss: 1.7564
Epoch 9/100
15685/15685 [==============================] - 151s 10ms/step - loss: 1.5454 - val_loss: 1.7274
Epoch 10/100
15685/15685 [======

In [18]:
generated = model.generate(contexts[0, :], 500, 0.5)
print(tokenizer.decode(generated.numpy())[seq_len:])

e the be present.

QUEEN ELIZABETH:
No, the call me the say to our grave to the own they father be so and the spill as the in dear of by the promise the send him for the with the have means to be consent when and to for my come other to be the dead.

First Servingman:
Thou will not provost to the words and would Call the me;
And the not shall the will not for the for the deed.

STANLEY:
Hark your poor of the droolw.

CLARENCE:
The shall one send me say and we did consent not more to the provost,
